In [159]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, K2Score, BicScore
from pgmpy.sampling.Sampling import BayesianModelSampling
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.readwrite.BIF import BIFWriter

def JSD(P, Q):
    _P = P / np.linalg.norm(P, ord=1)
    _Q = Q / np.linalg.norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

In [140]:
data = pd.DataFrame.from_csv('crash_sample_2018.csv', index_col=None)
del data['Unnamed: 0']
nodes = list(data.columns)
cat_to_num = {'AvgSpeed' : {'low' :0, 'high':1}, \
              'Country' : {'US':0, 'UK':1, 'Europe':2}, \
              'DangerLvl' : {'low':0, 'high':1}, \
              'NoAccidents' : {'low':0, 'medium':1, 'high':2}, \
              'NoFatalities' : {'low':0, 'medium':1, 'high':2}, \
              'NoJourneys' : {'low':0, 'medium':1, 'high':2}, \
              'PoliceActivity' : {'regular':0, 'increased':1}, \
              'RoadCond' : {'bad':0,'good':1}, \
              'Season' : {'winter':0, 'spring':1, 'summer':2, 'fall':3}, \
              'Weather' : {'bad':0, 'good':1}, \
              'Weekend' : {'working':0, 'weekend':1, 'holiday':2},
             }

num_to_cat = {}
for k1,v1 in cat_to_num.items():
    num_to_cat[k1] = { v2 : k2 for k2, v2 in v1.items()}

/Users/Luke/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [141]:
# select rows with missing data only
# data[data.isnull().any(axis=1)]

# get count of each combination in the data
# data.groupby(nodes).size()

# replace strings with numbers and back
assert data.equals(data.replace(cat_to_num).replace(num_to_cat))

In [142]:
train_mask = np.random.rand(len(data)) < 0.7
train = data[train_mask]
test = data[~train_mask]

In [143]:
baseline = BayesianModel()
baseline.add_nodes_from(nodes[1:])
baseline.add_edges_from([
    ('Country', 'Weather'),
    ('Country', 'RoadCond'),
    ('RoadCond', 'NoAccidents'),
    ('RoadCond', 'AvgSpeed'),
    ('NoAccidents', 'NoFatalities'),
    ('NoAccidents', 'PoliceActivity'),
    ('Weekend', 'NoJourneys'),
    ('Season', 'Country'),
    ('NoJourneys', 'RoadCond'),
    ('Weather', 'DangerLvl'),
])

In [144]:
baseline.fit(train)
BicScore(test).score(baseline)

-5000.655831762782

In [145]:
for i in range(0, 10):
    pred = train[train.isnull().any(axis=1)].apply(handle_missing, axis=1)
    complete_data = pd.concat([train[~train.isnull().any(axis=1)], pred])
    baseline.fit(complete_data)
    print(BicScore(test).score(baseline))

-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


-5000.655831762782


In [146]:
BicScore(test).score(baseline)

-5000.655831762782

In [147]:
join_prob = train.groupby(nodes).size() / len(train)

In [148]:
baseline.fit(train)

In [149]:
print(baseline.get_cpds('AvgSpeed'))
print(baseline.get_cpds('Country'))
print(baseline.get_cpds('DangerLvl'))
print(baseline.get_cpds('NoAccidents'))
print(baseline.get_cpds('NoFatalities'))
print(baseline.get_cpds('NoJourneys'))
print(baseline.get_cpds('PoliceActivity'))
print(baseline.get_cpds('RoadCond'))
print(baseline.get_cpds('Season'))
print(baseline.get_cpds('Weather'))
print(baseline.get_cpds('Weekend'))

╒════════════════╤═══════════════╤════════════════╕
│ RoadCond       │ RoadCond(bad) │ RoadCond(good) │
├────────────────┼───────────────┼────────────────┤
│ AvgSpeed(high) │ 0.5           │ 0.5            │
├────────────────┼───────────────┼────────────────┤
│ AvgSpeed(low)  │ 0.5           │ 0.5            │
╘════════════════╧═══════════════╧════════════════╛
╒═════════════════╤════════════════════╤════════════════════╤════════════════════╤════════════════════╕
│ Season          │ Season(fall)       │ Season(spring)     │ Season(summer)     │ Season(winter)     │
├─────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│ Country(Europe) │ 0.3333333333333333 │ 0.3333333333333333 │ 0.3333333333333333 │ 0.3333333333333333 │
├─────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│ Country(UK)     │ 0.3333333333333333 │ 0.3333333333333333 │ 0.3333333333333333 │ 0.3333333333333333 │
├───────────

In [150]:
sampler = BayesianModelSampling(baseline)

In [151]:
sample = sampler.forward_sample(10000)

In [152]:
join_prob_model = sample.groupby(nodes).size() / len(sample)
join_prob_model = join_prob_model.to_frame()
join_prob_model.columns = ['model']

In [153]:
join_prob = join_prob.to_frame()
join_prob.columns = ['orig']

In [154]:
to_compare = join_prob.join(join_prob_model, how='outer').fillna(0)

In [155]:
entropy(to_compare.model, to_compare.orig)

inf

In [156]:
hc = HillClimbSearch(train.replace(cat_to_num))
localMax = hc.estimate(start=baseline.copy())
localMax.fit(train.replace(cat_to_num))

In [157]:
BicScore(test).score(localMax)

-1232.9908713561385

In [163]:
writer = BIFWriter(localMax)
writer.write_bif(filename='model.bif')